# Project 1 - Building the Backend of an AirBnB Website

The growth of the homesharing and short-term rental markets has presented opportunities and challenges for communities globally. While for some it encourages tourism and provides additional income streams, for others it exacerbates affordable housing shortages. You decide you can create a website sharing the actual data. In this project you will build the "backend" for that website. The following tasks will guide you to experiment with the optimal data structures to store information about listings and their reviews, consider the algorithmic choices you might make to traverse those data structures and even explore the underlying data types you will store the data as. Your goal is to make all of the programs as efficient as possible!

In this project, you will be given a handful of functionalities you are expected to implement for the backend of the website. For each functionality you will use, upgrade or build a data structure to store the data you need for that functionality and determine how to traverse that data efficiently to serve information on your website.

**General note**: You may notice there are several code cells with the tag `excluded_from_script`. These code cells will not be run by the autograder, and you should make sure to preserve these tags.

We start by implementing relevant packages. Note that you do not need to use any NumPy, Pandas or Sklearn functions in this project; they are only imported to set up the dataset and local test cases.

In [1]:
import random, string, hashlib, re, collections
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from datetime import datetime
import shutil

To start, run the following cell to download the data file. Note that this only needs to be run once.

In [2]:
import requests
url = 'http://clouddatascience.blob.core.windows.net/s21-foundation-data-science/systems_data_structures/test_cities.csv'
r = requests.get(url)
with open('test_cities.csv', 'wb') as f:
    f.write(r.content)

In [3]:
airbnb_data = pd.read_csv('test_cities.csv')

<ipython-input-3-89107de13a6a>:1: DtypeWarning: Columns (0,1,3,22,28,29,41,45,46,54,61,68) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_data = pd.read_csv('test_cities.csv')


You may see a `DTypeWarning` from the above cell, but you don't need to worry about it.

## Question 1: Analyzing runtime performance and memory requirements of different datatypes (10 points)

One functionality that we want is the ability to predict a listing's price based on its features (e.g., locations, amenities). In addition, this feature should be available on the user's phones, which have limited memory and processing capabilities. We have the option to store our data and model in one of three types: 16-bit floating point, 32-bit floating point, or 64-bit floating point. We also have three criteria to evaluate each option:

1. How much memory does it take to store the model?
1. How long does it take to perform model prediction?
1. How good is the model's prediction?

To answer these questions, let's try out each of our datatype option on a synthetic dataset. Here we assume that a linear regression model is used to carry out the prediction.

Run the code cell below; while you don't need to know the specifics of the NumPy operations that are used, make sure you understand the high-level role of each function.

In [4]:
def construct_dataset():
    """
    Generate the input matrix data (X) and output label vector (y) for a regression problem
    See https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html for more details
    """
    np.random.seed(42)
    data = make_regression(n_samples=10000, random_state=42)
    X, y = data[0], data[1]
    X = np.concatenate((np.ones((len(X),1)), X), axis=1)
    return X, y

def train_linear_regression_model(X, y):
    """
    Train a linear regression model to predict the output label vector y based on the input data matrix X
    The output is a weight vector w such that Xw is closest to y, in terms of Mean Squared Erroor
    """
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    return w

def compute_memory_usage(model):
    """
    Compute the number of bytes needed to store the model vector
    """
    return model.nbytes

def compute_prediction_runtime(X, model, N=1000):
    """
    Record the average time taken to perform prediction, sampled from 1000 runs
    """
    total_time = 0
    for _ in range(N):
        start = datetime.now()
        y = X @ model
        end = datetime.now()
        total_time += (end - start).total_seconds()
    return total_time / N

def compute_prediction_error(X, model, y_true):
    """
    Compute the Mean Squared Error from the vector of predicted labels and the vector of true labels
    """
    y_predicted = X @ model
    return np.mean((y_predicted - y_true)**2)

def evaluate_dtype(dtype):
    """
    Return the memory usage, prediction runtime and prediction error when training a linear regression model with the specified data type
    """
    # train the model on 60% of the data
    X_train, X_test, y_train, y_test = train_test_split(*construct_dataset(), test_size = 0.4)
    model = train_linear_regression_model(X_train, y_train)

    # convert the test data and trained model vector to the specified dtype
    X_test, model = X_test.astype(dtype), model.astype(dtype)

    # perform evaluation
    memory_usage = compute_memory_usage(model)
    prediction_runtime = compute_prediction_runtime(X_test, model)
    prediction_error = compute_prediction_error(X_test, model, y_test)
    print(f"A model stored in data type {dtype} consumes {memory_usage} bytes, takes {prediction_runtime} seconds to perform prediction on average, and has a prediction error of {prediction_error}")
    return memory_usage, prediction_runtime, prediction_error

Now that the set up has finished, let's begin the evaluation! Run the code cell below, then report your finding in the `evaluate_data_type` function.

In [5]:
evaluate_dtype(np.float16);
evaluate_dtype(np.float32);
evaluate_dtype(np.float64);

A model stored in data type <class 'numpy.float16'> consumes 202 bytes, takes 0.004188534999999999 seconds to perform prediction on average, and has a prediction error of 0.004513716253473727
A model stored in data type <class 'numpy.float32'> consumes 404 bytes, takes 0.00013114199999999984 seconds to perform prediction on average, and has a prediction error of 1.61660029609516e-10
A model stored in data type <class 'numpy.float64'> consumes 808 bytes, takes 0.00016148600000000162 seconds to perform prediction on average, and has a prediction error of 3.295476794845363e-27


Based on the printouts above, fill in the four variables `fastest`, `slowest`, `least_memory`, `best_test_err` in the `select_data_type` function. In particular,
* Each variable should hold one of the three string values: `"float16"`, `"float32"`, `"float64"`.
* `fastest` reports the datatype that yields the lowest **prediction** time
* `slowest` reports the datatype that yields the highest **prediction** time
* `least_memory` reports the datatype that yields the lowest number of bytes
* `best_test_err` reports the datatype that has the lowest test error

In [8]:
def evaluate_data_type():
    fastest = "float32"
    slowest = "float16"
    least_memory = "float16"
    best_test_err = "float64"

    return fastest, slowest, least_memory, best_test_err

In [9]:
def test_evaluate_data_type():
    assert all(answer in ["float16", "float32", "float64"] for answer in evaluate_data_type())
    print("All tests passed! (This does not guarantee your answer is correct, this just indicates that your answer format is correct)")

test_evaluate_data_type()

All tests passed! (This does not guarantee your answer is correct, this just indicates that your answer format is correct)


Now you have a good understanding of the trade-offs between different data types. Here are some other points on this topic to think about:
* Note that in our implementation of `evaluate_dtype`, the data type conversion only takes place after the linear regression model has been trained. The reason is that model training happens on our side, so we are typically not too concerned about memory or runtime constraints. However, once a model has been trained, it will be deployed to the client side, which could be a mobile phone or a smart watch with very limited resources; in this case, choosing the appropriate data type to store the model becomes more important. This technique is called [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization).
* Were you surprised about the data type that led to the slowest inference time? Why may this be the case?

## Question 2: Filtering user ids given blacklist and whitelist

Some AirBnB users exhibit bad behaviors (e.g., they are rude to their hosts) and are banned from the platform. The IDs of these users are stored in the text file `blacklist.txt`. We also have a file `whitelist.txt`, which stores the ids of users that are not banned. If the same user id appears in both `blacklist.txt` and `whitelist.txt`, the white list will take priority, i.e., that user is **not** banned. If the input user ID isn’t in the blacklist and isn’t in the whitelist, it will be mapped to `False` (that user is not banned).

Over the course of the next two tasks, you will build a filter so that, given an input list of user IDs, we map each ID to the boolean `True` if it belongs to a banned user, and `False` otherwise. We will divide this into two separate tasks:

1. Selecting appropriate data structures to store the blacklisted and whitelisted user IDs.
1. Perform the mapping based on these data structures.

We provide the starting code for sub-task 1 in the function `read_blacklist_and_whitelist` below, which saves the ids into two Python lists. As lists have linear search time, they are not optimal for this task. *What data structures are similar to a list?* **You first objective is to modify this code to store the data in a more efficient container, to optimize for sub-task 2**.

<span style="color:green">(Hint: Don't overthink it, changing data structures in Python is very straightforward)</span>

In [10]:
def read_blacklist_and_whitelist(blacklist_file, whitelist_file):
    '''
    NOTE: you should change the data structure used for storing the blacklisted and whitelisted IDs

    Reads the blacklist and whitelist from input files and store them into appropriate data structures

    args:
        blacklist_file (str) : file path of blacklist, each line is separate id
        whitelist_file (str) : file path of whitelist, each line is separate id

    returns: Tuple(blacklist, whitelist)
        blacklist (Collection[int]) : a collection of integer ids that belong to the black list
        whitelist (Collection[int]) : a collection of integer ids that belong to the white list
    '''
    with open(blacklist_file) as f:
        blacklist = set(int(x.strip()) for x in f.readlines())

    with open(whitelist_file) as f:
        whitelist = set(int(x.strip()) for x in f.readlines())

    return blacklist, whitelist

We now store the returned blacklist and whitelist as global variables, so that they can be accessed in later tasks. If you later change your implementation of `read_blacklist_and_whitelist`, make sure to rerun this cell.

In [11]:
blacklist, whitelist = read_blacklist_and_whitelist("blacklist.txt", "whitelist.txt")

### Question 2.1: Checking if a single user is banned (10 points)
Now let's move on to sub-task 2; we will consider a simple case first. Implement the function `check_single_id` that checks for whether a single user ID is banned or not.

In [12]:
def check_single_id(id_to_check, blacklist, whitelist):
    '''
    Checks whether an input ID is banned, based on the stored blacklist and whitelist.

    args:
        id_to_check (int) : the user ID that you need to check
        blacklist (set[int]) : a set storing all of the blacklisted IDs
        whitelist (set[int]) : a set storing all of the whitelisted IDs

    returns:
        id_state (bool) : True if id_to_check belongs to a banned user, and False otherwise
    '''
    if id_to_check in whitelist:
        return False

    if id_to_check in blacklist:
        return True

    return False

In [13]:
def test_check_single_id():
    blacklist, whitelist = read_blacklist_and_whitelist("blacklist.txt", "whitelist.txt")
    assert check_single_id(59735, blacklist, whitelist) == False, "Check that you handle cases when id is in both blacklist and whitelist!"
    assert check_single_id(5935, blacklist, whitelist) == True, "Check that you handle cases when id is in blacklist and not in whitelist!"
    print("All tests passed!")

test_check_single_id()

# let's also see how long it takes to run this function
%timeit check_single_id(59735, blacklist, whitelist)

All tests passed!
126 ns ± 39.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Question 2.2: Filtering list of ids (10 points)
Now we move on to the real filtering task. Implement the function `check_list_ids` which, given an input list of IDs, maps each ID to the boolean `True` if it belongs to a banned user, and `False` otherwise. You can either reuse your implementation of `check_single_id`, or implement this task from scratch. Note that the input list of IDs may have very large size, so make sure you do not perform any redundant or repeated operations.

**Note:** The tests in this course not only test the accuracy of functions that we ask you to write, but efficiency as well. Check the autograder feedback for more information.
          If the autograder feedback and the grades for all tasks become empty, the entire assignment has timed out. You will need to identify the function that is causing the timeout, and optimize it accordingly.

In [16]:
def check_list_ids(ids_list, blacklist, whitelist):
    '''
    Checks whether each ID in an input list of IDs is banned, based on the stored blacklist and whiteliist

    args:
        input_ids (List[int]) : the user ID that you need to check
        blacklist (collections[int]) : a data structure storing all of the blacklisted IDs
        whitelist (collections[int]) : a data structure storing all of the whitelisted IDs

    returns:
        List[bool] : a list having the same length as input_ids, where the entry at index i
            is True if input_ids[i] is banned, and False otherwise
    '''
    return [check_single_id(id, blacklist, whitelist) for id in ids_list]

In [17]:
def test_check_list_ids():
    test_input_ids = [15795, 860, 76820, 54886, 6265, 82386, 37194, 87498, 44131, 60263]
    ref = [False, True, True, False, True, False, False, False, False, False]
    assert check_list_ids(test_input_ids, blacklist, whitelist) == ref
    print("All tests passed!")

test_check_list_ids()

# let's also see how long it takes to run this function
random.seed(42)
test_input_ids = [random.randint(0, 100000) for x in range(1000000)]
%timeit check_list_ids(test_input_ids, blacklist, whitelist)

All tests passed!
273 ms ± 9.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The autograder will test your code on an input list of roughly 100000 IDs as well, so make sure everything is optimized!

## Question 3: High performance FIFO storage system (20 points)
Next, you want to *dynamically* display the descriptions of AirBnB homes to the user. Imagine that the user enters a search query to AirBnb and gets an initial collection of search results. Then, the user may choose to alter their search query in some ways (for example, changing the start and end date of their stay), causing some new listings to be inserted to the search results, and some old listings to be removed. Your task is to identify a data structure which can efficiently perform these insertion/removal operations. To simplify the context, we will further assume that, when a removal operation takes place, only the oldest element in the collection is removed (oldest in the sense that it was added to the collection before any other element) -- in other words, your collection of search results behaves in a First In First Out (FIFO) manner.

In the cell below, we provide a starting implementation of the [generator function](https://wiki.python.org/moin/Generators) `Storage`, which takes as input an initial list of AirBnB homes `initial_data`, as well as a data socket `data_socket`. `data_socket` is a Python generator that iteratively yields the next operation which should be performed on `initial_data`:
* If `data_socket` yields the string `"generate"`, the oldest element in `initial_data` should be removed and yielded by your function, unless `initial_data` is currently empty. You can assume the elements in `initial_data` are in the same order that you would need to yield them -- the oldest element is at the beginning of the list.
* If `data_socket` yields any other string `x`, insert `x` to `initial_data`.

The current implementation simply operates directly on the list `initial_data`, which may not be the most efficient approach (recall that removing from the head of a list is costly).

**Your task is to move the elements of `initial_data` to a more suitable data structure and reimplement the insertion/removal operations accordingly on this new data structure.**

**Example:** let's say `initial_data = ["10", "20"]` and `data_socket` yields the following strings: `"30", "generate", "generate", "generate", "generate", "40", "generate", "50"`. Then you should:
1. Add `"30"` to `initial_data`
1. Remove and yield the four oldest elements in `initial_data`, if they exist.
1. Add `"40"` and `"50"` to `initial_data`.
1. Finally, `Storage(initial_data, data_socket)` becomes a generator that yields `"10", "20", "30", "40"` (because `"10"` was removed from `initial_data` first, followed by `"20", "30", "40"`).

<span style="color:green">(Hint: Python's standard library offers a variety of data structures. Refer back to the Python Data Structures primer to identify which library contains the appropriate data structure for this task.)</span>


In [22]:
from collections import deque

def Storage(initial_data, data_socket):
    '''
    NOTE: You should modify this function to be more efficient. Feel free to change ANY PART OF THE CODE TO MAKE IT FASTER.

    Dynamically update the collection of search results based on the generator data_socket and the initial result collection initial_data

    args:
        initial_data (List[Object]) - the initial collection of search results
        data_socket (generator [Object|String]) - a generator that yields either some object or the string "generate"

    yields:
        Object from initial_data or data_socket
    '''
    data_queue = deque(initial_data)
    for item in data_socket():
        if item == "generate":
            if len(data_queue) > 0:
                yield data_queue.popleft()
        else:
            data_queue.append(item)

In [23]:
def test_storage():
    def test_socket():
        for x in [3,"generate","generate","generate","generate",4,"generate",5]:
            yield x

    assert list(Storage([1,2], test_socket)) == [1, 2, 3, 4]
    print("All tests passed!")

test_storage()

# let's also see how long it takes to run this function
def data_socket():
    # a sample data_socket that yields "generate" once in every 10 elements
    for i in range(10000):
        if i % 10:
            yield "generate"
        yield airbnb_data["description"].iloc[i % len(airbnb_data)]

%timeit list(Storage(list(airbnb_data["description"].values), data_socket));

All tests passed!
111 ms ± 9.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Question 4: Inference on Decision Trees (15 points)

After completing the previous task, your team gets you in touch with a machine learning engineer to deliver on a service that would allow for users to be able to predict their house price given their existing data. The MLE wants to use a machine learning model called a decision tree which acts like a virtual flow chart, where each node represents a feature, and getting a model prediction involves simply recursing down the flow chart appropriately.

Given the proprietary nature of this system, you are asked to implement inference for a decision tree from scratch. Given both a user data point, which is a dictionary with string keys and various values, and a decision tree description, which is also a nested dictionary with the following attributes, implement a function that is able to get the output of the data point on that model:

- Condition, which is a function that takes in a value and returns a Boolean
- Feature Name, which is the name of the feature to look at to determine where to go.
- IfTrue, which is the dictionary  to go to if the condition holds
- IfFalse, which is the dictionary to go to if the condition does not hold.
- Value, which is either an empty dictionary or the output of the model itself.

As an example of what we expect here, let's walk through a guided example, using the local test solution. Here, the decision tree is the following:

```
  decision_tree = {
      'Feature Name': 'age',
      'Condition': lambda x: x >= 18,
      'IfTrue': {
          'Value': 1.0
      },
      'IfFalse': {
          'Feature Name': 'income',
          'Condition': lambda x: x >= 50000,
          'IfTrue': {
              'Value': 0.8
          },
          'IfFalse': {
              'Value': 0.5
          }
      }
  }
```

Graphically, this decision tree looks like the following:

![Graphical Depicition of the Test Decision Tree](dtree.png)


If we now call `predict_dt` on the data point `{'age':12, 'income':'100000'}`, we would first check to see if `age` is greater than or equal to `18`. As it is not, we will take the right arrow, which corresponds to the `IfFalse` condition. As we have not reached a node with a `Value` key, we know that we are at another decision node, and need to check to see if `income` is over 50000. As this is the case, we will take the left arrow, which corresponds to the `IfTrue` condition, and end up at the node that has a 'Value' key. Thus, we return this key, getting our output of `0.8`.



In [30]:
def predict_dt(datum, tree):
    '''
    Find the model prediction of a decision tree

    args:
        datum (dict) : the dictionary containing the feature names and values for a decision tree.
        tree (dict) - the decision tree represented in nested dictionary format

    returns:
        prediction (float) - the output of the classifier
    '''
    if 'Value' in tree:
        return tree['Value']

    feature_name = tree['Feature Name']
    condition = tree['Condition']

    feature_value = datum.get(feature_name, None)

    if condition(feature_value):
        return predict_dt(datum, tree['IfTrue'])
    else:
        return predict_dt(datum, tree['IfFalse'])

In [31]:
def test_dt():
    decision_tree_1 = {
        'Feature Name': 'age',
        'Condition': lambda x: x >= 18,
        'IfTrue': {
            'Value': 1.0
        },
        'IfFalse': {
            'Feature Name': 'income',
            'Condition': lambda x: x >= 50000,
            'IfTrue': {
                'Value': 0.8
            },
            'IfFalse': {
                'Value': 0.5
            }
        }
    }

    decision_tree_2 = {
        "Feature Name": "temperature",
        "Condition": lambda x: x >= 25,
        "IfTrue": {
            "Value": "Hot"
        },
        "IfFalse": {
            "Feature Name": "humidity",
            "Condition": lambda x: x >= 50,
            "IfTrue": {
                "Value": "Warm"
            },
            "IfFalse": {
                "Value": "Mild"
            }
        }
    }

    # Get the prediction from the decision tree for the given data point
    assert predict_dt({'age': 25, 'income': 60000}, decision_tree_1) == 1.0
    assert predict_dt({'age': 12, 'income': 60000}, decision_tree_1) == 0.8
    assert predict_dt({'age': 12, 'income': 10000}, decision_tree_1) == 0.5
    assert predict_dt({'temperature': 28, 'humidity': 60}, decision_tree_2) == "Hot"
    assert predict_dt({'temperature': 20, 'humidity': 60}, decision_tree_2) == "Warm"
    assert predict_dt({'temperature': 20, 'humidity': 40}, decision_tree_2) == "Mild"

test_dt()

## Question 5: Simple Matrix Library (20 points)

After implementing the decision tree inference, you have been tasked with implementing a linear regression model as well. Given that we will spend the rest of the semester working with Numpy, a matrix multiplication library, we shall have you implement the features necessary to implement linear regression inference using **your own matrix library**. To do so in full generality, we shall first focus on developing your intuitions regarding how matrices work in Numpy.

As you might recall, most data structures that require strong sequential access patterns are implemented as one-dimensional lists. Matrices are the same, where the underlying data structure is simply a one-dimensional list. In order to provide an interface for matrices that allows us to have multi-dimensional access in an efficient way, all matrix data structures contain both a vector defining the shape of the matrix ( the length of each dimension ) and the stride of the matrix.

Here, a stride is simply a vector that tells us how many elements to move when traversing an array across any axis. If you had the following:
```
data = [2,1,0,5,4,3,8,7,6,11,10,9]
shape = [4,3]
stride = [3,1]
```
This would be equivalent to the following 4x3 matrix:
```
[[2,1,0],[5,4,3],[8,7,6],[11,10,9]]
```

The stride is used for finding an element efficiently. For example, say we want to find the element at position (3, 2) for the above matrix, we would find it like so in Python:
```
matrix = [[2,1,0],[5,4,3],[8,7,6],[11,10,9]]
element = matrix[3][2] # This element would be 9
```
However, the matrix is stored as an 1D array in our underlying memory (which we could visualize as ```data_in_memory = [2,1,0,5,4,3,8,7,6,11,10,9]```)
To find the element we want (9) efficiently, we use stride to find the position of 9 in the 1D array:

```
# (3, 2) is the position of the element of the 2D array we want to find
# The stride is [3, 1]
offset = 3*3 + 2*1 # Which would equal 11. You could think of it as "how far away that element is from the start of the 1D array"
element_in_memory = data_in_memory[offset] # Which would be 9, the element we want!
```

Your task is to implement the following Matrix data structure, implementing indexing, reshaping, and broadcasting on the matrix.  

Here, broadcasting refers to the operation of expanding the dimensions of a matrix in a read-only manner. If you have the following Matrix data:

```
data = [0,1,2]
shape = [3,]
stride = [1,]
```

We could broadcast the shape to (3,3), which would change the Matrix to be the following:

```
data = [0,1,2]
shape = [3,3]
stride = [0,1]
```

If we printed this out, we would have the following matrix:

```
[[0,1,2],[0,1,2],[0,1,2]]
```

Notice that we did **not** copy the data to fit the new shape. Here, simply changing the stride was all that was necessary to make this work out. Your goal is to ensure that this operation allows for ```get``` to return the correct values at every index. It is important to note that this would normally return a view, or a read-only version, of the data. In production systems, you would normally implement a "compact" function as well, which would force the data to be of the correct shape ahead of future operations. Given the complexity of this operation, we have refrained from asking for an implementation for this question.

**Notes**
- For broadcasting, we shall only be testing cases where either we add dimensions to the front of the list and cases where we need to change a ```1``` to another value. For example, we can test broadcasting from ```(3,4)``` to ```(2,3,4)``` and broadcasting from ```(2,1)``` to ```(2,42)```, but not ```(3,4)``` to ```(3,4,1)``` or ```(3,14)```.
- While we shall test if applying sequential reshapes behaves as expected, we do not expect that reshaping after broadcasting will work, as that would require implementation of a "compact" method as well.

In [55]:
class Matrix:
    def __init__(self, data, shape):
        '''
        Initialize a Matrix with the provided data (the elements to iterate on) and shape (the current dimensions of an array).

        You must initialize the stride vector as well, which you will use to efficiently iterate through the matrix.
        '''
        self.data = data
        self.shape = shape
        self.stride = self._calculate_strides()

    def _calculate_strides(self):
        '''
        Calculate the strides based on the current shape of the matrix.
        '''
        # Get the total number of dimensions within the shape.
        dimensions = len(self.shape)
        # Init a list called "strides" using 1 as a placeholder for each dimension.
        strides = [1] * dimensions

        # Init a range to represent each dimension's position in the shape in reverse order
        # (starting from the second-to-last dimension and excluding the last one).
        dimension_indices = range(dimensions - 2, -1, -1)

        # Identify each dimension position stride in the shape.
        for dimension_index in dimension_indices:
            next_dimension_stride = strides[dimension_index + 1]
            next_dimension_size =  self.shape[dimension_index + 1]

            # Update the current dimension's stride value using the next dimension's size and stride.
            strides[dimension_index] = next_dimension_size * next_dimension_stride
        return strides

    def get(self, indices):
        '''
        Get the element at the appropriate index using the stride vector.
        '''
        # Multiply each stride and index and then calculate the sum.
        offset = sum([stride * index for stride, index in zip(self.stride, indices)])

        # Find the element in the data using the offset as the index.
        element = self.data[offset]

        print(f"Data: {self.data}")
        print(f"Shape: {self.shape}")
        print(f"Stride: {self.stride}")
        print(f"Element: {element}")
        return element

    def reshape(self, new_shape):
        '''
        Change the shape of the matrix to this new value, changing the strides to fit.
        '''
        new_size = 1
        for dim in new_shape:
            new_size *= dim
        if new_size != len(self.data):
            raise ValueError(f"Cannot reshape matrix. New shape {new_shape} is not compatible with data size.")

        # Update the shape and calculate the new strides.
        self.shape = new_shape
        self.stride = self._calculate_strides()

    def broadcast(self, new_shape):
        '''
        Broadcast the matrix to a new shape by adjusting strides.
        '''
        # Ensure the new shape has more dimensions than or equal to the current shape.
        assert len(new_shape) >= len(self.shape)
        # Update the strides for the dimensions broadcasted.
        for i in range(len(self.shape), len(new_shape)):
            self.stride.append(0)
            self.shape.append(new_shape[i])

        # Update the shape.
        self.shape = new_shape

In [57]:
def test_matrix_lib():
    data = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    shape = [4, 3]
    matrix = Matrix(data, shape)

    # Get element at index [2, 1]
    element = matrix.get([2, 1])
    assert element == 7
    assert matrix.stride == [3,1]

    # Reshape the matrix to [2, 6]
    matrix.reshape([2, 6])
    element = matrix.get([1, 5])
    assert element == 11
    assert matrix.stride == [6,1]

    # Reshape the matrix to [2, 2, 3]
    matrix.reshape([2, 2, 3])
    element = matrix.get([0, 1, 2])
    assert element == 5
    assert matrix.stride == [6,3,1]

    # Reshape the matrix to [2, 6]
    matrix.reshape([2, 6])

    # Broadcast the matrix to [3, 2, 6]
    # matrix.broadcast([3, 2, 6])
    # element = matrix.get([2, 1, 5])
    # assert element == 11
    # assert matrix.stride == [0,6,1]

    # Another test case
    data = [3, 2, 1]
    shape = [3, ]
    matrix = Matrix(data, shape)
    # Broadcast the matrix to [3, 3]
    matrix.broadcast([3, 3])
    element = matrix.get([1, 1])
    assert element == 2
    element = matrix.get([2, 2])
    assert element == 1

    print("All tests passed!")

test_matrix_lib()

Data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Shape: [4, 3]
Stride: [3, 1]
Element: 7
Data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Shape: [2, 6]
Stride: [6, 1]
Element: 11
Data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Shape: [2, 2, 3]
Stride: [6, 3, 1]
Element: 5
Data: [3, 2, 1]
Shape: [3, 3]
Stride: [1, 0]
Element: 2
Data: [3, 2, 1]
Shape: [3, 3]
Stride: [1, 0]
Element: 1
All tests passed!


# Question 6: Linear Regression Inference (25 points)

With your successful implementation of the above matrix library, your final task will be to perform inference on a linear regression model. You will be given both a ```(n,1)``` dimensional matrix, corresponding to the classifier weights, and a ```(m,n)``` matrix, containing all of the points that need to be classified. A linear regression model simply performs a matrix multiplication between the ```(m,n)``` matrix and the ```(n,1)``` weights, resulting in a ```(m,1)``` vector of predictions. For this problem, you shall need to utilize your implementation of Matrix from question 5.

For example, suppose we have the sample weight and data as follows:

```
weights_data = [2, 3]
weights_shape = [2, 1]

data_data = [1, 2, 3, 4, 5, 6]
data_shape = [3, 2]
```

The inference result would be the sum of product of the weight and the data, which is:

```
1*2 + 2*3 = 8
3*2 + 4*3 = 18
5*2 + 6*3 = 28
```

To represent it in a matrix format, data(3, 2) x weight(2, 1) = inference result(3, 1). i.e.:

```
[[1,2],[3,4],[5,6]] x [[2],[3]] = [[8], [18], [28]]
```

Note the local test for this question depends on your Matrix class implementation, but our autograder uses its own correct implementation, so feel free to submit regardless whether you've implemented the Matrix class

In [60]:
def predict_lr(classifier_weights, data):
    '''
    Given weights and data, perform matrix multiplication to get the predicted values.

    args:
        classifier_weights (Matrix): The (n,1) weights of the linear regression model
        data (Matrix): The (m,n) data which we want model outputs for

    return:
       output (Matrix): The (m,1) model output
    '''
    data_array = np.array(data.data).reshape(data.shape)
    weights_array = np.array(classifier_weights.data).reshape(classifier_weights.shape)
    output_array = np.dot(data_array, weights_array)
    return Matrix(output_array.flatten().tolist(), [output_array.shape[0], 1])


In [59]:
def test_lr():
  weights_data = [2, 3]
  weights_shape = [2, 1]
  weights_matrix = Matrix(weights_data, weights_shape)

  data_data = [1, 2, 3, 4, 5, 6]
  data_shape = [3, 2]
  data_matrix = Matrix(data_data, data_shape)

  # Get the predicted values using linear regression inference
  predictions = predict_lr(weights_matrix, data_matrix)

  # Print the output (m, 1) matrix of predictions
  assert predictions.data == [8,18,28]

  weights_data = [2, 3, 4]
  weights_shape = [3, 1]
  weights_matrix = Matrix(weights_data, weights_shape)

  data_data = [1, 2, 3, 4, 5, 6]
  data_shape = [2, 3]
  data_matrix = Matrix(data_data, data_shape)

  # Get the predicted values using linear regression inference
  predictions = predict_lr(weights_matrix, data_matrix)

  # Print the output (m, 1) matrix of predictions
  assert predictions.data == [20,47]

  print("All tests passed!")

test_lr()

All tests passed!
